In [1]:
import re
from io import StringIO
import pandas as pd

In [2]:
def parse_x1(num, header=['Center Number', 'Atomic Number', 'Atomic Type', 'X', 'Y', 'Z']):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    so = re.finditer('(S|s)tandard orientation', outtxt)
    for s in so:
        pass
    last_so = s.end()
    delim = re.finditer('---------------------------------------------------------------------', outtxt[last_so:])
    
    i = 0
    for d in delim:
        if i == 1:
            first_delim_end = d.end()
        if i == 2:
            second_delim_start = d.start()
        i += 1
        
    return pd.read_csv(StringIO(outtxt[last_so+first_delim_end:last_so+second_delim_start]), delim_whitespace=True, header=None, names=header)
    

In [3]:
def parse_x2_occ(num):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    alpha_occ = re.finditer('(A|a)lpha  occ. eigenvalues --', outtxt)

    all_rows = []
    rows = []
    for occ in alpha_occ:
        l = outtxt[occ.end():occ.end()+51]
        match = re.search('A', l)
        if match is not None:
            print(match)
            rows.append(list(filter(lambda a: a != '', re.split('\s', outtxt[occ.end():occ.end()+match.start()-2]))))
            all_rows.append(rows)
            rows = []
        else:
            rows.append(list(filter(lambda a: a != '', re.split('\s', l))))
            
    return pd.DataFrame(all_rows[len(all_rows)-1])

In [4]:
def parse_x2_virt(num):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    alpha_occ = re.finditer('(A|a)lpha virt. eigenvalues --', outtxt)

    all_rows = []
    rows = []
    le = 0
    occ = next(alpha_occ, None)
    while occ is not None:
        le = occ.end()
        l = outtxt[occ.end():occ.end()+51]
        occ = next(alpha_occ, None)
        if occ is None or occ.start() - le > 55:
            rows.append(list(filter(lambda a: a != '' and a != 'Condensed' and a != 'to', re.split('\s', l))))
            all_rows.append(rows)
            rows = []
        else:
            rows.append(list(filter(lambda a: a != '', re.split('\s', l))))
            
    return pd.DataFrame(all_rows[len(all_rows)-1])

In [5]:
num = 186
parse_x2_virt(num)

,0,1,2,3,4
0,-0.01175,-0.00764,0.00569,0.00732,0.00984
1,0.02463,0.02755,0.03310,0.04443,0.05332
2,0.05382,0.05700,0.05978,0.07290,0.07788
3,0.08411,0.08975,0.09183,0.10507,0.10976
4,0.11311,0.11955,0.12656,0.12985,0.13207
5,0.13256,0.13585,0.14681,0.15094,0.15374
6,0.15902,0.16396,0.16622,0.16711,0.16746
7,0.17045,0.17849,0.18338,0.18638,0.19257
8,0.19658,0.19876,0.20197,0.20908,0.21234
9,0.21500,0.21976,0.22665,0.23165,0.23659
